![SIMEM_logo_1.png](../../assets/SIMEM_logo_1.png)


[SiMEM](https://www.simem.co/)

## Objetivos de este Notebook

* Explicar cómo se puede consultar información de una variable predeterminada.

## Índice

1. [Importación de la clase VariableSIMEM](#section1)

2. [Consultar el listado de variables disponibles](#section2)

3. [Buscar el código de una variable](#section3)

4. [Consultar datos de la variable](#section4)


<a id='section1'></a>
### Importación de la clase VariableSIMEM

En la siguiente celda se importa el objeto VariableSIMEM, que contiene las funciones necesarias para cumplir los objetivos anteriormente mencionados.

In [1]:
import sys

# !{sys.executable} -m pip install pydataxm    # Instala la librería pydatasimem.
from pydataxm.pydatasimem import VariableSIMEM # Importa la libreria que fue instalada con pip install pydataxm
import plotly.graph_objects as go
import pandas as pd

<a id='section2'></a>
### Consultar el listado de variables disponibles dentro de la clase

En la siguiente celda se muestra como consultar el listado de variables con el fin de buscar el código de la variable deseada.

In [2]:
lisatado_variables = VariableSIMEM.get_collection() # Obtiene el listado de variables que se pueden usar dentro de VariableSIMEM
lisatado_variables.head()                           # Imprime los primeros 5 registros del listado de variables

,CodigoVariable,Nombre,Dimensiones
0,AportesHidricosEnergia,Aportes hídricos de las series hidrológicas ex...,"[CodigoSerieHidrologica, RegionHidrologica, Fe..."
1,AportesHidricosMasa,Aportes hídricos de las series hidrológicas,"[CodigoSerieHidrologica, RegionHidrologica, Fe..."
2,AportesHidricosMasaPSS95,Aportes hídricos para un 95% de Probabilidad d...,"[CodigoSerieHidrologica, RegionHidrologica, Fe..."
3,CantidadVentasContratosSICEP,Cantidad ventas en contratos del SICEP,"[Version, FechaHora]"
4,CapacidadAsignada,Capacidad asignada al proyecto con reserva,[FechaActualizacion]


<a id='section3'></a>
### Buscar el código de una variable de interes

En la siguiente celda se muestra como consultar el código de una variable a través de su nombre

In [3]:
nombre_variable = 'Precio' # Nombre de la variable que se desea buscar
cod_variable = lisatado_variables.query("Nombre.str.lower().str.contains(@nombre_variable.lower())")['CodigoVariable'].values # Busca el nombre dentro del listado de variables
print(cod_variable)        # Imprime los códigos de las variables que coinciden con el nombre

['MPO_Int' 'MPO_Nal' 'MPO_Tie' 'PAR_Ofertado' 'PB_Int' 'PB_Nal' 'PB_Tie'
 'PMaximoCargoConfi' 'POf_Original' 'PONE_ExpCol' 'PPBO' 'PPBOGReal'
 'PrecioEscasez' 'PrecioEscasezActivacion' 'PrecioEscasezInferior'
 'PrecioEscasezPonderado' 'PrecioEscasezPonderadoAgente'
 'PrecioEscasezSuperior' 'PrecioMarginalEscasez'
 'PrecioOfertaArranqueParadaCOP' 'PrecioOfertaArranqueParadaUSD'
 'PrecioOfertaConfiguracion' 'PTB']


Una vez obtenido el código de la variable de interés, se debe inicializar una instancia de la clase VariableSIMEM con este y dos fechas de interés, para poder obtener la información de la variable.

In [4]:
cod_variable = 'PrecioEscasez' # Código de la variable deseada
fecha_inicio = '2024-01-01'    # Fecha inicial de los datos
fecha_fin = '2024-12-31'       # Fecha final de los datos

variable = VariableSIMEM(cod_variable, fecha_inicio, fecha_fin) # Se inicializa el objeto VariableSIMEM

<a id='section4'></a>
### Consultar datos de la variable

En este caso se desea con sultar la información del `Precio de Escasez` durante el 2024

In [5]:
data = variable.get_data() # Obtiene los datos de la variable
data

****************************************************************************************************
Initializing object
The object has been initialized with the dataset: "Precios de escasez del Cargo por Confiabilidad"
****************************************************************************************************
Inicio consulta sincronica
Creacion url: 0.0018765926361083984
Extraccion de registros: 3.627126932144165
End of data extracting process
****************************************************************************************************


,CodigoVariable,CodigoDuracion,Valor
Fecha,,,
2024-12-01,PrecioEscasez,P1M,798.7259
2024-11-01,PrecioEscasez,P1M,811.9274
2024-10-01,PrecioEscasez,P1M,751.3103
2024-09-01,PrecioEscasez,P1M,785.0686
2024-08-01,PrecioEscasez,P1M,818.3814
2024-07-01,PrecioEscasez,P1M,810.7083
2024-06-01,PrecioEscasez,P1M,766.5341
2024-05-01,PrecioEscasez,P1M,793.3028
2024-04-01,PrecioEscasez,P1M,742.6505


### Graficar la serie de tiempo de la variable

Teniendo ya la información consultada del 2024 para el `Precio de Escasez` graficaremos como es su comportamiento a lo largo del tiempo.

In [6]:
df_reset = data.reset_index() # Quita el índice de fecha para porder usarla al momento de gráficar

df_reset["Fecha"] = pd.to_datetime(df_reset["Fecha"]) # Convierte la columna a Fecha a un tipo de dato datetime
df_reset.sort_values(by="Fecha", inplace=True)        # Ordena los datos según la fecha

fig = go.Figure() # Crea una figura vacía de Plotly para agregar trazos (gráficas)

fig.add_trace(go.Scatter(
 x=df_reset["Fecha"], # Asigna la columna 'Fecha' al eje X
 y=df_reset["Valor"], # Asigna la columna 'Valor' al eje Y
 mode='lines',        # Especifica que se dibuje una línea (sin marcadores)
 name="Valor"         # Nombre de la serie que aparecerá en la leyenda
))

fig.update_layout(
 title="Serie de tiempo",           # Título principal del gráfico
 xaxis_title='Fecha',               # Etiqueta del eje X
 yaxis_title='Valor',               # Etiqueta del eje Y
 xaxis=dict(tickformat='%d-%m-%y'), # Formato de fecha para las marcas del eje X (día-mes-año)
 template='plotly_white'            # Estilo visual del gráfico (fondo blanco y diseño limpio)
)

fig.show() # Muestra la gráfica directamente en el notebook